<h2>Лабораторная работа №2 по курсу "Проектирование интеллектуальных систем"</h2>

<p><b>Выполнил:</b> Саврасов П.А. группа ИУ5-24М</p>

<h3>Задание</h3>
    <p>Создать логистическую регрессию для классификации набора данных MNIST.</p>
    <p>Создать нейронную сеть с 5 полносвязными слоями для классификации набора данных MNIST с
количеством нейронов в слоях от первого до пятого - (200,100,60,30,10).</p>
</ul>

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

Загрузим датасет MNIST и рассмотрим его формат

In [26]:
(xTrain, yTrain), (xTest, yTest) = mnist.load_data()

print(xTrain.shape, yTrain.shape)
print(xTest.shape, yTest.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


Как видно, формат датасета (обучающей выборки) следующий: <br>
<ul>
    <li>60000 образцов изображений</li>
    <li>28 пикселей в высоту</li>
    <li>28 пикселей в ширину</li>
</ul>
То есть это трёхмерный массив данных. Такой формат не подходтит для обучения нейросетей, требуется изименить форму датасета на двухмерный массив. Число образцов оставим неизменным, а двумерный массив пикселей преобразуем в одномерный: 

In [27]:
xTrain = tf.reshape(xTrain, (60000, 28 * 28))
print(xTrain.shape)
xTest = tf.reshape(xTest, (10000, 28 * 28))
print(xTest.shape)

(60000, 784)
(10000, 784)


Рассмотрим формат одного элемента массива:

In [28]:
print(xTrain[0].dtype)
print(xTrain[0][200:250])

<dtype: 'uint8'>
tf.Tensor(
[  0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82  82  56
  39   0   0   0   0   0   0   0   0   0   0   0   0  18 219 253 253 253
 253 253 198 182 247 241   0   0   0   0   0   0   0   0], shape=(50,), dtype=uint8)


Формат элемента является 8 битным беззнаковым целым, диапазон значений которого лежит в промежутке от 0 до 255. Такой формат для обучения нейросестей не самый удачный. Произведём масштабирование и преобразуем диапазон от 0 до 1. При этом нужно учесть, что сначала нужно преобразовать тип данных в float32

In [29]:
xTrain = tf.cast(xTrain, dtype = tf.float32) / 255
xTest = tf.cast(xTest, dtype = tf.float32) / 255
print(xTrain[0].dtype)
print(xTrain[0][200:250])


<dtype: 'float32'>
tf.Tensor(
[0.         0.         0.         0.19215687 0.93333334 0.99215686
 0.99215686 0.99215686 0.99215686 0.99215686 0.99215686 0.99215686
 0.99215686 0.9843137  0.3647059  0.32156864 0.32156864 0.21960784
 0.15294118 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.07058824 0.85882354 0.99215686 0.99215686 0.99215686
 0.99215686 0.99215686 0.7764706  0.7137255  0.96862745 0.94509804
 0.         0.         0.         0.         0.         0.
 0.         0.        ], shape=(50,), dtype=float32)


Датасет преобразован и готов для обучения моделей <br>
Начнём с обучения логистической регрессии

In [79]:
model = tf.keras.Sequential(
    [
        layers.InputLayer(input_shape=(28*28)),
        layers.Dense(10, activation=tf.nn.log_softmax)
    ]
)


In [80]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.SGD(learning_rate = 0.5),
    metrics = ['accuracy']
)

In [81]:
model.fit(xTrain, yTrain, batch_size=32, epochs=5, verbose=2)
model.evaluate(xTest, yTest, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 2s - loss: 0.3758 - accuracy: 0.8908
Epoch 2/5
1875/1875 - 2s - loss: 0.3210 - accuracy: 0.9094
Epoch 3/5
1875/1875 - 2s - loss: 0.3092 - accuracy: 0.9127
Epoch 4/5
1875/1875 - 2s - loss: 0.3012 - accuracy: 0.9155
Epoch 5/5
1875/1875 - 2s - loss: 0.2972 - accuracy: 0.9174
313/313 - 0s - loss: 0.2945 - accuracy: 0.9199


[0.2944934666156769, 0.9199000000953674]

Теперь обучим нейронную сеть следующими параметрами:
<ul>
    <li>Входной слой:   784</li>
    <li>Скрытый слой 1: 200</li>
    <li>Скрытый слой 2: 100</li>
    <li>Скрытый слой 3: 60</li>
    <li>Скрытый слой 4: 30</li>
    <li>Выходной слой:  10</li>
</ul>    
Сначала расмотрим упрощённый вариант на основе Sequential API

In [50]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(28*28)),
        layers.Dense(200, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(30, activation='relu'),
        layers.Dense(10, activation='relu')
    ]
)

 Установим модели функцию потерь и оптимизатор (способ минимизации функции потери)

In [51]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.SGD(learning_rate = 0.5),
    metrics = ['accuracy']
)

Обучим модель со следующими параметрами:<br>
Размер батча (порция данных): 32 образца<br>
Число эпох: 5

In [52]:
model.fit(xTrain, yTrain, batch_size=32, epochs=5, verbose=2)
model.evaluate(xTest, yTest, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 4s - loss: 1.3936 - accuracy: 0.5317
Epoch 2/5
1875/1875 - 4s - loss: 0.7025 - accuracy: 0.7898
Epoch 3/5
1875/1875 - 4s - loss: 0.1642 - accuracy: 0.9573
Epoch 4/5
1875/1875 - 4s - loss: 0.1280 - accuracy: 0.9669
Epoch 5/5
1875/1875 - 4s - loss: 0.1111 - accuracy: 0.9706
313/313 - 0s - loss: 0.1453 - accuracy: 0.9667


[0.1453077793121338, 0.96670001745224]

Теперь повторим тоже самое, но уже с применением Functional API

In [53]:
inpLayer = keras.Input(shape=(28*28))
hidLayer1 = layers.Dense(200, activation='relu')(inpLayer)
hidLayer2 = layers.Dense(100, activation='relu')(hidLayer1)
hidLayer3 = layers.Dense(60, activation='relu')(hidLayer2)
hidLayer4 = layers.Dense(30, activation='relu')(hidLayer3)
outLayer = layers.Dense(10, activation='relu')(hidLayer4)

model = keras.Model(inputs=inpLayer, outputs=outLayer)

In [54]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.SGD(learning_rate = 0.5),
    metrics = ['accuracy']
)

In [55]:
model.fit(xTrain, yTrain, batch_size=32, epochs=5, verbose=2)
model.evaluate(xTest, yTest, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 4s - loss: 1.1223 - accuracy: 0.6493
Epoch 2/5
1875/1875 - 4s - loss: 0.8117 - accuracy: 0.7653
Epoch 3/5
1875/1875 - 4s - loss: 0.7794 - accuracy: 0.7744
Epoch 4/5
1875/1875 - 4s - loss: 0.7640 - accuracy: 0.7793
Epoch 5/5
1875/1875 - 4s - loss: 0.7529 - accuracy: 0.7824
313/313 - 0s - loss: 0.7632 - accuracy: 0.7790


[0.7632175087928772, 0.7789999842643738]

<h3>Контрольные вопросы</h3>
<ol>
    <li>
        <h4>Что такое Variable?</h4>
        <p>	В TensorFlow для хранения значений модели существует специальный тип tf.Variable. В отличие от других Tensor объектов которые заново обновляются при каждом запуске сессии, переменные (Variable) хранят фиксированное значение в графе. Это является важным, т.к. при текущее значение переменной влияет на вывод в вычисляемой итерации. Как и другие Tensor объекты, переменные можно использовать как входные значения в графе.</p>
    </li>
    <li>
        <h4>Что такое placeholder?</h4>
        <p>	Для добавления входных данных извне модели в TensorFlow используется специальный тип - плейсхолдер (placeholder). Плейсхолдер можно представить в виде пустой переменной который будет заполняться данными позже. Сперва их используют для создания графа и заполняют данными при выполнении сессии.</p>
    </li>
    <li>
        <h4>Что такое функция потерь?</h4>
        <p>Функция потерь(стоимости) – используется в качестве метрики для определения качества модели. Это расстояние(разница) между предсказанием модели и истинным значением входного вектора.</p>
    </li>
    <li>
        <h4>Какие другие названия функции потери?</h4>
        <p>Функция стоимости.</p>
    </li>
    <li>
        <h4>Зачем нужна функция потери?</h4>
        <p>Функция потерь(стоимости) – используется в качестве метрики для определения качества модели. Это расстояние(разница) между предсказанием модели и истинным значением входного вектора.</p>
    </li>
    <li>
        <h4>Как запустить обучение модели?</h4>
        <p><b>Для TensorFlow 1:</b> В метод tf.Session().run() передаем шаг градиентного спуска и значения для placeholder.</p>
        <p><b>Для TensorFlow 2:</b> В метод класса keras.Sequential fit передать данные о числе эпох, размеру батча, методу визуализации процесса обучения, и данных для обучения</p>
    </li>
    <li>
        <h4>Что делает tf.global_variables_initializer()?</h4>
        <p>Вызывается при вызове метода сессии .run() для создания в оперативной памяти области для хранения переменных и их исходных значений.</p>
    </li>
    <li>
        <h4>Что такое minibatch?</h4>
        <p>Небольшая порция примеров из общего датасета. Обычно объем данной подвыборки варьируется от 50 до 500 примеров.</p>
    </li>
    <li>
        <h4>Какие бывают активационные функции?</h4>
        <p>Логистическая, тангенсальная и ReLU (Rectified Linear Unit) активационные функции.</p>
    </li> 
</ol>